In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
#         print(os.path.join(dirname, filename))
        pass

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
url = 'paraphrase-mpnet-base-v2'

In [3]:
import torch

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [5]:
torch.cuda.empty_cache()

In [6]:
train_path_cov = "/kaggle/input/final-supplementary-data/final_train.json"
dev_path_cov   = "/kaggle/input/final-supplementary-data/final_val.json"
test_path_cov  = "/kaggle/input/final-supplementary-data/final_test.json"

In [7]:
train_samples_frame = pd.read_json(train_path_cov, lines=True)
train_samples_frame.head()

,id,image,labels,text,bb_dict,ent,bestLabel
0,covid_memes_18,covid_memes_18.png,"[somewhat harmful, individual]",Bernie or Elizabeth?\nBe informed.Compare them...,"[[0.6680887372013651, 0.632857142857142, 0.825...","[Elizabeth I, Politics, Meme, Media, Text, Ber...",presentation
1,covid_memes_19,covid_memes_19.png,"[somewhat harmful, organization]",Extending the\nBrexit deadline until\nOctober ...,[],"[Meme, Internet meme, r/PrequelMemes, Image, C...",literature review memes
2,covid_memes_252,covid_memes_252.png,[not harmful],kwai\ngkwa 0964\n#nnevvy\napplause to Thais fr...,"[[0.522067189216613, 0.207160174846649, 0.4088...","[Freddie Highmore, Finding Neverland]",freddie highmore finding neverland
3,covid_memes_255,covid_memes_255.png,[not harmful],"So, I order this\nfoce mask to\nprotect ogains...",[],"[Meme, Coronavirus, Funny meme, Virus, Image, ...",corona memes cartoon
4,covid_memes_20,covid_memes_20.png,"[somewhat harmful, individual]",best candidate for\nJA\n2020\njoe biden\nKamal...,"[[0.47865853658536506, 0.32835820895522305, 0....","[T-shirt, successful, Shirt]",shoulder


In [8]:
train_samples_frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3013 entries, 0 to 3012
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         3013 non-null   object
 1   image      3013 non-null   object
 2   labels     3013 non-null   object
 3   text       3013 non-null   object
 4   bb_dict    3013 non-null   object
 5   ent        3013 non-null   object
 6   bestLabel  3013 non-null   object
dtypes: object(7)
memory usage: 164.9+ KB


In [9]:
test_samples_frame = pd.read_json(test_path_cov, lines=True)
test_samples_frame.head()

,id,image,labels,text,bb_dict,ent,bestLabel
0,covid_memes_5425,covid_memes_5425.png,[not harmful],gwen\n@gwenervi\ndis gon be trump tomorrow aft...,[],"[Resident Evil 5, Albert Wesker, Uroboros Viru...",uroboros test subject
1,covid_memes_5426,covid_memes_5426.png,[not harmful],Armani\n@historyofarmani\nBiden after hearing ...,"[[0.071942446043165, 0.2734375, 0.393884892086...","[Human, Human behavior, Motivational speaker, ...",photo caption
2,covid_memes_5429,covid_memes_5429.png,[not harmful],MESSAGE FROM TRUMP TO\nCOVID-19\nLEAVE NOW OR ...,"[[0.41454545454545405, 0.125683060109289, 0.64...","[Meme, Birthday, Photograph, Image, Funny meme...",photo caption
3,covid_memes_5430,covid_memes_5430.png,[not harmful],COVID-19 STARTED DURING HIS TERM\nSOIT SHOULD ...,"[[0.257246376811594, 0.19125683060109203, 0.54...","[Meme, Politics, Coronavirus disease 2019, Cor...",photo caption
4,covid_memes_5434,covid_memes_5434.png,[not harmful],TRUMPS RESPONSE TO COVID-19\nUMP\nTAYM\nINCLUD...,"[[0.25703868269920305, 0.167230486869812, 0.49...","[official, Photo caption, EFL Championship]",trumpette


In [10]:
test_samples_frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354 entries, 0 to 353
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         354 non-null    object
 1   image      354 non-null    object
 2   labels     354 non-null    object
 3   text       354 non-null    object
 4   bb_dict    354 non-null    object
 5   ent        354 non-null    object
 6   bestLabel  354 non-null    object
dtypes: object(7)
memory usage: 19.5+ KB


In [11]:
val_samples_frame = pd.read_json(dev_path_cov, lines=True)
val_samples_frame.head()

,id,image,labels,text,bb_dict,ent,bestLabel
0,covid_memes_2069,covid_memes_2069.png,[not harmful],Coronavirus\nVaccine\nInjection only\n5 ml Sto...,"[[0.556944444444444, 0.6181640625, 0.838888888...","[COVID-19 vaccine, Coronavirus disease 2019, M...",
1,covid_memes_2075,covid_memes_2075.png,[not harmful],THE NEW\nINDIAN EXPRESS\nRussia has developed ...,"[[0.41388888888888803, 0.391746749576031, 0.55...","[Transport, Motor vehicle]",vehicle
2,covid_memes_2080,covid_memes_2080.png,[not harmful],Ssrfan @Ssrfan478780364 - 2h\nThe owners of Zo...,"[[0.44473684210526304, 0.270588235294117, 0.64...","[John Terry, Face, Forehead]",john terry crying
3,covid_memes_2108,covid_memes_2108.png,[not harmful],Alexa Simone thatssokrahe 14h\nMy coworker sto...,"[[0.619043707847595, 0.40359678864479004, 0.20...","[Funny meme, Meme, funny, Coronavirus disease ...",multimedia
4,covid_memes_2117,covid_memes_2117.png,[not harmful],Mele Gdl\n*s the sonual tension between Plizer...,"[[0.230057120323181, 0.650536954402923, 0.2507...","[Coronavirus disease 2019, World Health Organi...",us health authorities tracking new covid 19 va...


In [12]:
val_samples_frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177 entries, 0 to 176
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         177 non-null    object
 1   image      177 non-null    object
 2   labels     177 non-null    object
 3   text       177 non-null    object
 4   bb_dict    177 non-null    object
 5   ent        177 non-null    object
 6   bestLabel  177 non-null    object
dtypes: object(7)
memory usage: 9.8+ KB


In [13]:
!pip install git+https://github.com/UKPLab/sentence-transformers

  Cloning https://github.com/UKPLab/sentence-transformers to /tmp/pip-req-build-vt4693w7
  Running command git clone --filter=blob:none --quiet https://github.com/UKPLab/sentence-transformers /tmp/pip-req-build-vt4693w7
  Resolved https://github.com/UKPLab/sentence-transformers to commit c5f93f70eca933c78695c5bc686ceda59651ae3b
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=126132 sha256=6760385edbd98409bcfc2aed614200c66d53d1dc1f41eb2269a25848467e9e07
  Stored in directory: /tmp/pip-ephem-wheel-cache-rfm894uz/wheels/da/46/bf/66a68ee8dce733df91a66fc155274e0a38283f3db647b5ea0d
Successfully built sentence-transformers


In [14]:
from sentence_transformers import SentenceTransformer
model_sent_trans = SentenceTransformer(url)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [15]:
class HarmemeMemesDatasetAug2(torch.utils.data.Dataset):
    """Uses jsonl data to preprocess and serve 
    dictionary of multimodal tensors for model input.
    """

    def __init__(
        self,
        data_path,
        img_dir,
        split_flag=None,
        balance=False,
        dev_limit=None,
        random_state=0,
    ):

        self.samples_frame = pd.read_json(
            data_path, lines=True
        )
        self.samples_frame = self.samples_frame.reset_index(
            drop=True
        )
        
    def __len__(self):
        """This method is called when you do len(instance) 
        for an instance of this class.
        """
        return len(self.samples_frame)

    def __getitem__(self, idx):
        """This method is called when you do instance[key] 
        for an instance of this class.
        """
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_id = self.samples_frame.loc[idx, "id"]
        img_file_name = self.samples_frame.loc[idx, "image"]

        cur_ent_rep_list = []
        cur_ent_list = self.samples_frame.loc[idx, "ent"]
        best_label = self.samples_frame.loc[idx, "bestLabel"]
        label_feature = torch.tensor(model_sent_trans.encode(best_label)).to(device)
        
        if len(cur_ent_list):
            for item in cur_ent_list:
                cur_ent_rep = torch.tensor(model_sent_trans.encode(item)).to(device)
                cur_ent_rep_list.append(cur_ent_rep)
            text_drob_feature = torch.mean(torch.vstack(cur_ent_rep_list), axis=0)
        else:
            text_drob_feature = torch.tensor(model_sent_trans.encode(self.samples_frame.loc[idx, "text"])).to(device)
        
        final_feature = torch.cat((label_feature, text_drob_feature))

        return final_feature

In [16]:
from torch.utils.data import DataLoader

In [17]:
data_dir_cov = "/kaggle/input/harmeme/MINI_PROJECT_2/Harm-C/datasets/memes/defaults/images"

hm_dataset_train = HarmemeMemesDatasetAug2(train_path_cov, data_dir_cov, 'train')
dataloader_train = DataLoader(hm_dataset_train, batch_size=64, shuffle=True, num_workers=0)

hm_dataset_test = HarmemeMemesDatasetAug2(test_path_cov, data_dir_cov, 'test')
dataloader_test = DataLoader(hm_dataset_test, batch_size=64, shuffle=True, num_workers=0)

hm_dataset_val = HarmemeMemesDatasetAug2(dev_path_cov, data_dir_cov, 'val')
dataloader_val = DataLoader(hm_dataset_val, batch_size=64, shuffle=True, num_workers=0)

In [18]:
def obtain_tensor(dataloader):
    tensors = []
    for batch in dataloader:
        tensors.append(batch)

    concatenated_tensor = torch.cat(tensors, dim=0)
    concatenated_tensor.shape
    return concatenated_tensor

In [ ]:
train_tensor = obtain_tensor(dataloader_train)

In [ ]:
test_tensor = obtain_tensor(dataloader_test)

In [19]:
val_tensor = obtain_tensor(dataloader_val)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [20]:
val_tensor.shape

torch.Size([177, 1536])

In [21]:
def shaper(input_tensor):
    linear_layer = torch.nn.Linear(1536, 768).to(device)
    output_tensor = linear_layer(input_tensor)
    return output_tensor

In [22]:
val_tensor_2 = shaper(val_tensor)

In [ ]:
train_tensor_2 = shaper(train_tensor)
test_tensor_2 = shaper(test_tensor)

In [23]:
val_tensor_2.shape

torch.Size([177, 768])

In [ ]:
torch.save(train_tensor_2, 'Train_text_Drob_Feature_MiniLM.pt')
torch.save(test_tensor_2, 'Test_text_Drob_Feature_MiniLM.pt')
torch.save(val_tensor_2, 'Val_text_Drob_Feature_MiniLM.pt')

In [ ]:
val_ENT = torch.load("/kaggle/input/harmeme/MINI_PROJECT_2/harmeme_saved_feat_ROIENT/harmeme_ENT_MOMENTA/cov/harmeme_cov_harmfulness/harmeme_cov_val_ent.pt")

In [ ]:
val_ENT.shape